<a href="https://colab.research.google.com/github/vamshap/PySpark-Challenges/blob/main/Tournament.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, lit, coalesce

# write a code to show the each team how many matches played , matches won , matches lost
# Create Spark session
spark = SparkSession.builder.appName("ICCWorldCup").getOrCreate()

# Create the data
icc_world_cup_data = [
    ('India', 'SL', 'India'),
    ('SL', 'Aus', 'Aus'),
    ('SA', 'Eng', 'Eng'),
    ('Eng', 'NZ', 'NZ'),
    ('Aus', 'India', 'India')
]

# Define schema
columns = ["Team_1", "Team_2", "Winner"]

# Create DataFrame
icc_world_cup_df = spark.createDataFrame(icc_world_cup_data, schema=columns)

# Step 1: Calculate matches as Team_1
team_1_df = icc_world_cup_df.groupBy("Team_1").agg(count("Team_1").alias("MatT1"))

# Step 2: Calculate matches as Team_2
team_2_df = icc_world_cup_df.groupBy("Team_2").agg(count("Team_2").alias("MatT2"))

# Step 3: Calculate wins
win_df = icc_world_cup_df.groupBy("Winner").agg(count("Winner").alias("WinMat"))

# Step 4: Join DataFrames and calculate matches played, won, and lost
result_df = (
    team_1_df.join(team_2_df, team_1_df.Team_1 == team_2_df.Team_2, "left_outer")
    .join(win_df, team_1_df.Team_1 == win_df.Winner, "left_outer")
    .select(
        team_1_df.Team_1.alias("Team"),
        (coalesce(team_1_df.MatT1, lit(0)) + coalesce(team_2_df.MatT2, lit(0))).alias("Matches_Played"),
        coalesce(win_df.WinMat, lit(0)).alias("Matches_Win"),
        (
            (coalesce(team_1_df.MatT1, lit(0)) + coalesce(team_2_df.MatT2, lit(0)))
            - coalesce(win_df.WinMat, lit(0))
        ).alias("Matches_Lost")
    )
)

# Show the result
result_df.show()

+-----+--------------+-----------+------------+
| Team|Matches_Played|Matches_Win|Matches_Lost|
+-----+--------------+-----------+------------+
|   SL|             2|          0|           2|
|India|             2|          2|           0|
|  Eng|             2|          1|           1|
|   SA|             1|          0|           1|
|  Aus|             2|          1|           1|
+-----+--------------+-----------+------------+

